In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169#, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.utils import multi_gpu_model
import tensorflow as tf

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
size=(137,137)

def transformImg(img):
    img=255-img
    mu = cv2.moments(img, False)
    x, y= mu["m10"]/mu["m00"] , mu["m01"]/mu["m00"]
    M = np.float32([[1,0,(236/2)-x],[0,1,(137/2)-y]])
    img = cv2.warpAffine(img,M,(img.shape[1],img.shape[0]))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(img,-1,kernel)
    img = cv2.filter2D(img,-1,kernel)
    img = cv2.resize(img, (size[0],size[1]))
    #img = preprocess_input(img)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [8]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, i):
        _imgs=self.X[i * self.batch_size:(i + 1) * self.batch_size,:,:]
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img))
        
        imgs=np.asarray(imgs).astype(np.float32)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[i * self.batch_size:(i + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training and np.random.random()<0.8:
        #if self.training:
            r= np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            alpha=np.random.rand(imgs.shape[0])
            imgs=np.tile(alpha,(3,*size,1)).T*imgs+np.tile((1-alpha),(3,*size,1)).T*imgs2
            grapheme=np.tile(alpha,(168,1)).T*grapheme+np.tile((1-alpha),(168,1)).T*grapheme2
            vowel=np.tile(alpha,(11,1)).T*vowel+np.tile((1-alpha),(11,1)).T*vowel2
            consonant=np.tile(alpha,(7,1)).T*consonant+np.tile((1-alpha),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
            
        if self.training:
            imgs = [randomErase(img) for img in imgs]
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [9]:
def getMultiXceptionBased():
    model =  Xception(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    #x = Dense(1024, activation="relu")(x)
    x = Dropout(0.3)(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
del df

In [11]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="Xception-epoch{epoch:04}.h5")

In [ ]:
train_gen = DataLoader(X_train, y_train, training=True, batch_size=64)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=64)
model = getMultiXceptionBased()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[2.,1.,1.])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=60, callbacks=[early_stopping, checkpoint], workers=multiprocessing.cpu_count(), use_multiprocessing=True)
model.save("multixception_without3.h5")

83689472/83683744 [==============================] - 13s 0us/step
compiled
Epoch 1/60
2825/2825 [==============================] - 975s 345ms/step - loss: 6.9324 - dense_1_loss: 2.6366 - dense_2_loss: 1.0280 - dense_3_loss: 0.6313 - dense_1_acc: 0.5354 - dense_2_acc: 0.7674 - dense_3_acc: 0.8427 - val_loss: 1.3632 - val_dense_1_loss: 0.4966 - val_dense_2_loss: 0.1519 - val_dense_3_loss: 0.1391 - val_dense_1_acc: 0.8645 - val_dense_2_acc: 0.9589 - val_dense_3_acc: 0.9602
Epoch 2/60
2825/2825 [==============================] - 939s 333ms/step - loss: 5.5673 - dense_1_loss: 2.0922 - dense_2_loss: 0.8508 - dense_3_loss: 0.5321 - dense_1_acc: 0.6800 - dense_2_acc: 0.8191 - dense_3_acc: 0.8757 - val_loss: 0.9989 - val_dense_1_loss: 0.3305 - val_dense_2_loss: 0.1193 - val_dense_3_loss: 0.1132 - val_dense_1_acc: 0.9178 - val_dense_2_acc: 0.9737 - val_dense_3_acc: 0.9725
Epoch 3/60
2825/2825 [==============================] - 939s 333ms/step - loss: 5.1672 - dense_1_loss: 1.9378 - dense_2_loss:

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 940s 333ms/step - loss: 4.1996 - dense_1_loss: 1.5456 - dense_2_loss: 0.6665 - dense_3_loss: 0.4420 - dense_1_acc: 0.7442 - dense_2_acc: 0.8577 - dense_3_acc: 0.8996 - val_loss: 0.8110 - val_dense_1_loss: 0.1757 - val_dense_2_loss: 0.0604 - val_dense_3_loss: 0.0616 - val_dense_1_acc: 0.9525 - val_dense_2_acc: 0.9858 - val_dense_3_acc: 0.9833
Epoch 9/60
1156/2825 [===========>..................] - ETA: 8:57 - loss: 3.9830 - dense_1_loss: 1.4639 - dense_2_loss: 0.6339 - dense_3_loss: 0.4213 - dense_1_acc: 0.7573 - dense_2_acc: 0.8702 - dense_3_acc: 0.9067